In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Ipynb_importer
from utils_x import *
from parameters_x import *

all_col = ['unique_symbol', 'date_time', 'close', 'turnover', 'open', 'high', 'low', 'volume',"use_tag", "adjustment", "adjustment_cumsum"]
shift_col = ['close', 'turnover', 'open', 'high', 'low', 'volume', 'wap', "adjustment_cumsum"]

# clean3 对数据进行 factor处理后 根据clean2标记的tag准备训练数据

In [ ]:
p_col = ['close', 'open', 'high', 'low']
v_col = ['turnover', 'volume', 'wap']

print("------------------- load data--------------------")
# data = pd.read_csv('index500_v2.csv', dtype=object)
data = pd.read_csv('index500_v2_adjustment.csv', dtype=object)
data["adjustment"] =  data["adjustment"].fillna(0)


print("------------------- pre data--------------------")
data = data[all_col].copy()
data['turnover'] = data['turnover'].apply(lambda x:np.float(x))
data['volume'] = data['volume'].apply(lambda x:np.float(x))
data['open'] = data['open'].apply(lambda x:np.float(x))
data['close'] = data['close'].apply(lambda x:np.float(x))
data['high'] = data['high'].apply(lambda x:np.float(x))
data['low'] = data['low'].apply(lambda x:np.float(x))
data['adjustment'] = data['adjustment'].apply(lambda x:np.float(x))
data['adjustment_cumsum'] = data['adjustment_cumsum'].apply(lambda x:np.float(x))
print("---------------- pre data step 1-----------------")


In [ ]:
data['wap'] = data['turnover']/data['volume']
gb = data.groupby(['unique_symbol'])['open']
data['t+1_open'] = gb.transform(lambda x: x.shift(-1))
data['t+2_open'] = gb.transform(lambda x: x.shift(-2))
print("------------- open shift done --------------------")

In [ ]:
gb = data.groupby(['unique_symbol'])['adjustment']
data['t+2_adjustment'] = gb.transform(lambda x: x.shift(-2))
data["y_open"] = data["t+2_open"] - data['t+2_adjustment']
data['return'] = data.apply(lambda x: (x['y_open'])/x['t+1_open']-1, axis=1)
print("------------- y data done --------------------")

In [ ]:
# data[data["unique_symbol"] == "000016"]["open"].mean()

In [ ]:
data = data[~(data['return'] > 0.25)].copy()
data = data[~(data['return'] < -0.25)].copy()


data = data[~data['return'].isnull()].copy()
gb = data.groupby(['date_time'])['return']
data['label'] = gb.transform(lambda x: (x-x.mean())/x.std())
data['rank_label'] = gb.transform(lambda x: pd.qcut(x.rank(method='first'), 10, labels=False, duplicates="drop"))

print(len(data))
print("---------------- shift data-------------------")
shift_day = 15
for col in shift_col:
    print(col)
    gb_col = "unique_symbol"
    data = nshift(data, gb_col, col, shift_day)
for col in p_col:
    adjust_col = "adjustment_cumsum"    
    data = norm_pcol(data, col, adjust_col, shift_day)
for col in v_col:
    data = norm_vcol(data, col, shift_day)

In [ ]:
data.to_csv("index500_v3_adjustment.csv")